In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import LSTM, Dense
from transformers import TFBertForSequenceClassification
import pandas as pd
import numpy as np
!pip install pandas transformers sklearn nltk
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel
import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data=pd.read_csv('/content/Data.csv')
data.head()

,ID,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [ ]:
# Check for null values in the dataset
null_values = data.isnull().sum()

# Display the columns with null values
print("Null Values in Each Column:")
print(null_values[null_values > 0])

# Drop rows with any null values
df_clean = data.dropna()

# Verify that null values have been removed
print(f"Rows after dropping nulls: {df_clean.shape[0]}")

# Drop rows where 'YourColumn' has null values
df = df_clean.dropna(subset=['text'])

# Display the filtered dataset
print(df.head())

# Optionally, check the number of rows remaining after dropping nulls
print(f"Rows remaining after dropping nulls: {df.shape[0]}")
df.head()



Null Values in Each Column:
Series([], dtype: int64)
Rows after dropping nulls: 6315
      ID                                              title  \
0   8476                       You Can Smell Hillary’s Fear   
1  10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2   3608        Kerry to go to Paris in gesture of sympathy   
3  10142  Bernie supporters on Twitter erupt in anger ag...   
4    875   The Battle of New York: Why This Primary Matters   

                                                text  label  
0  Daniel Greenfield, a Shillman Journalism Fello...      0  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...      0  
2  U.S. Secretary of State John F. Kerry said Mon...      1  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...      0  
4  It's primary day in New York and front-runners...      1  
Rows remaining after dropping nulls: 6315


,ID,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [ ]:
# Check for null values in the dataset
null_values = df.isnull().sum()

# Display the columns with null values
print("Null Values in Each Column:")
print(null_values[null_values > 0])


Null Values in Each Column:
Series([], dtype: int64)


In [ ]:
### Tokenization - using BERT ###

# Initialize BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text column
df['tokenized_text'] = df['text'].apply(lambda x: bert_tokenizer.tokenize(x))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
### Lematization ###

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply lemmatization on the tokenized text
df['lemmatized_text'] = df['tokenized_text'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])




In [ ]:
### Stop word Removal ###

# Get English stopwords
stop_words = set(stopwords.words('english'))

# Remove stopwords from the lemmatized text
df['cleaned_text'] = df['lemmatized_text'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])



In [ ]:
### Feature Extraction - for traditional ML models ###

# Convert the cleaned tokens back to text format for TF-IDF vectorization
df['cleaned_text_str'] = df['cleaned_text'].apply(lambda tokens: ' '.join(tokens))

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the cleaned text to generate TF-IDF features
tfidf_features = tfidf_vectorizer.fit_transform(df['cleaned_text_str'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the TF-IDF feature DataFrame
tfidf_df.head()



,00,000,10,100,11,12,13,14,15,150,...,yu,za,ze,zed,zero,zi,zing,zionist,zone,zu
0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.059977,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.03505,0.000000,0.0,0.037169,0.0,0.0,0.0,0.04089,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00000,0.042207,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
### Feature Extraction - for DL using BERT ###

# Initialize BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# Function to get BERT embeddings
def get_bert_embeddings(text):
    # Tokenize input text
    inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    # Get hidden states from BERT
    with torch.no_grad():
        outputs = bert_model(**inputs)
        hidden_states = outputs.hidden_states

    # Get the embeddings from the last hidden state
    embeddings = hidden_states[-1]
    return torch.mean(embeddings, dim=1).squeeze().numpy()

# Apply BERT embedding extraction on the cleaned text
df['bert_embeddings'] = df['cleaned_text_str'].apply(get_bert_embeddings)

# Create a new DataFrame for the embeddings
bert_embeddings_df = pd.DataFrame(df['bert_embeddings'].to_list())

# Display the BERT embeddings DataFrame
bert_embeddings_df.head()



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Import the necessary library
from sklearn.model_selection import train_test_split

# Split dataset into features (X) and labels (y)
X = df['text']
y = df['label']

# Split into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data, and transform the test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_curve, roc_curve, roc_auc_score
import seaborn as sns
import time
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


In [ ]:
# Split the dataset into features (X) and labels (y)
X = df['text'].tolist()
y = df['label'].tolist()

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data (BERT requires a specific input format)
train_encodings = bert_tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = bert_tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors='tf')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Load pre-trained BERT model for binary classification
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model with Adam optimizer and sparse categorical crossentropy loss (since this is a binary classification task)
# Use the 'adam' string to specify the optimizer instead of passing the optimizer object directly
bert_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Measure training latency
start_train = time.time()

# Train the model
history = bert_model.fit(
    train_encodings['input_ids'], np.array(y_train),
    validation_data=(test_encodings['input_ids'], np.array(y_test)),
    epochs=3,
    batch_size=16
)

end_train = time.time()
train_latency = end_train - start_train
print(f"Training Latency: {train_latency:.4f} seconds")


Epoch 1/3
 98/316 [========>.....................] - ETA: 1:19:43 - loss: 0.7810 - accuracy: 0.5166

KeyboardInterrupt: 

In [ ]:
# Measure prediction latency
start_predict = time.time()

# Get predictions on the test set
y_pred_probs = bert_model.predict(test_encodings['input_ids'])
y_pred = np.argmax(y_pred_probs.logits, axis=1)

end_predict = time.time()
predict_latency = end_predict - start_predict
print(f"Prediction Latency: {predict_latency:.4f} seconds")


KeyboardInterrupt: 

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


ValueError: Found input variables with inconsistent numbers of samples: [1263, 2526]

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Print the classification report
class_report = classification_report(y_test, y_pred, target_names=['Fake', 'Real'])
print(class_report)


In [ ]:
# Precision-recall curve
y_scores = tf.nn.softmax(y_pred_probs.logits, axis=1)[:,1].numpy()  # Get probabilities
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

# Plot the precision-recall curve
plt.figure(figsize=(6,4))
plt.plot(recall, precision, marker='.', label='BERT')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()


In [ ]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores)

# Plot the ROC curve
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, marker='.', label='BERT (AUC = %0.2f)' % roc_auc_score(y_test, y_scores))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()
